<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2111, done.
remote: Counting objects: 100% (929/929), done.
remote: Compressing objects: 100% (471/471), done.
remote: Total 2111 (delta 608), reused 686 (delta 449), pack-reused 1182
Receiving objects: 100% (2111/2111), 49.69 MiB | 7.49 MiB/s, done.
Resolving deltas: 100% (1386/1386), done.


# import 

In [2]:
from c620.F import F
from c620.config import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# functions

In [3]:
def check_two_data(d1,d2):
  if np.allclose(d1.values,d2.values):
    print('一樣')
  else:
    print('不一樣')

# demo data

In [4]:
tag = 'test001'
demo = joblib.load('./data/demo(real_data).pkl')
print(demo.keys())

icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']

icg_input

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c660_op', 'c670_op', 'c620_wt', 'c660_wt', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
2021-02-13,182.727005,2.0600,11.670,37.270,17.901134,2.3700,68.700,21.000,98.045372,0.6650,43.532,42.478,87.140,838.0,0.0,29.0,40.842743
2021-02-05,179.608627,2.1650,12.377,38.074,18.646059,3.0880,66.645,22.216,96.942879,0.7025,43.370,42.201,85.878,800.0,0.0,16.0,40.885593
2020-12-04,166.107956,2.1510,12.047,37.287,19.038383,3.4740,65.729,19.557,99.873688,0.6486,43.878,42.602,86.102,805.0,0.0,35.0,39.059292
2020-11-26,169.727284,1.3600,12.000,37.630,18.240871,3.0000,66.560,21.430,92.989864,0.6610,43.786,41.398,86.390,749.0,0.0,19.0,41.073098
2020-11-16,176.370055,1.8370,11.857,37.198,17.663765,2.6220,66.132,21.395,98.291358,0.5919,44.655,42.658,92.836,738.0,0.0,32.0,43.761401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-03,179.751343,1.3600,11.500,37.000,18.112499,2.1900,69.510,19.010,104.812675,0.5970,46.987,39.682,85.740,579.0,0.0,52.0,39.083649
2020-12-05,176.814285,1.9710,12.018,37.289,19.088993,3.3900,66.171,20.551,96.033325,0.6312,43.357,43.284,85.918,795.0,0.0,33.0,40.796818
2021-01-29,168.094742,2.1770,12.611,38.067,19.249239,2.3479,69.407,19.462,104.291405,0.5800,44.610,42.030,90.047,692.0,0.0,44.0,35.294071
2020-09-13,170.544997,1.9050,10.267,35.719,12.889589,2.8940,73.021,19.166,62.388201,0.5200,35.900,53.680,88.119,858.0,0.0,2.0,40.121723


In [5]:
idx1 = icg_input.index[0]
idx2 = icg_input.index[1]

# 1.確保給定不同的輸入 會有不同的輸出

In [6]:
f = F(config)
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
f._Post_processing = False

# 輸入1
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 輸入2
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f(icg_input.loc[[idx2]],c620_feed.loc[[idx2]],t651_feed.loc[[idx2]])

current Distillate Rate_m3/hr:0.24963613313555788 NA in Benzene_ppmw:980.0
current Distillate Rate_m3/hr:0.2512798362961229 NA in Benzene_ppmw:980.0


In [7]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 2.確保變更模式 推薦 和 試算 模式 有不同的輸出

In [8]:
# 試算模式
f.Recommended_mode = False #(True or False)
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 推薦模式
f.Recommended_mode = True #(True or False)
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

current Distillate Rate_m3/hr:0.24963613313555788 NA in Benzene_ppmw:980.0


In [9]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 確認icg_input 對應到後續系統的輸出值一致 主要有兩項要一致
* 1.benzene in c620 side
* 2.na in benzene

In [10]:
# 真實數據 試算模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = False #(True or False)
f.real_data_mode = True
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

            Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
2021-02-13                                              87.14               
            Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
2021-02-13                                              838.0     
            Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
2021-02-13                                          72.465913                                   


2021-02-13    1008.216586
dtype: float64

In [11]:
# 真實數據 推薦模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

            Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
2021-02-13                                              87.14               
            Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
2021-02-13                                              838.0     
current Distillate Rate_m3/hr:0.24963613313555788 NA in Benzene_ppmw:980.0
            Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
2021-02-13                                          70.263429                                   


2021-02-13    1034.569967
dtype: float64

In [12]:
# 模擬數據 試算模式
demo = joblib.load('./data/demo.pkl')
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
idx1 = icg_input.index[0]
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = False #(True or False)
f.real_data_mode = False
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
140-023                                                 70               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
140-023                                                980     
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023                                          74.690655                                   


140-023    942.814055
dtype: float64

In [13]:
# 模擬數據 推薦模式
demo = joblib.load('./data/demo.pkl')
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
idx1 = icg_input.index[0]
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
f.Recommended_mode = True #(True or False)
f.real_data_mode = False
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
naidx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]
c660_wt1.filter(regex='Sidedraw').iloc[:,naidx].sum(axis=1)*10000

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
140-023                                                 70               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
140-023                                                980     
current Distillate Rate_m3/hr:0.896723051205885 NA in Benzene_ppmw:980.0
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023                                          74.542048                                   


140-023    942.854657
dtype: float64

# 操作條件預測太準的問題

In [14]:
demo = joblib.load('./data/demo(real_data).pkl')
print(demo.keys())
idx = demo['icg_input'].sample(1).index[0]
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
f._linear_model = False
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f(demo['icg_input'].loc[[idx]],demo['c620_feed'].loc[[idx]],demo['t651_feed'].loc[[idx]])

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c660_op', 'c670_op', 'c620_wt', 'c660_wt', 'c670_wt'])
current Distillate Rate_m3/hr:0.4566823309521598 NA in Benzene_ppmw:980.0


In [15]:
pd.concat([c620_op.iloc[:,-2:],demo['c620_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
2020-10-09,172.972107,178.742218
2020-10-09,173.129485,187.076827


In [16]:
pd.concat([c660_op.iloc[:,-2:],demo['c660_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
2020-10-09,84.936203,88.626732
2020-10-09,85.169942,88.937150


In [17]:
pd.concat([c670_op.iloc[:,-2:],demo['c670_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
2020-10-09,180.975677,225.885849
2020-10-09,182.139600,239.521996


# 把操作模組改成線性模組

In [18]:
f.Recommended_mode = True #(True or False)
f.real_data_mode = True
f._linear_model = True
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f(demo['icg_input'].loc[[idx]],demo['c620_feed'].loc[[idx]],demo['t651_feed'].loc[[idx]])

current Distillate Rate_m3/hr:0.4566823309521598 NA in Benzene_ppmw:980.0


In [19]:
pd.concat([c620_op.iloc[:,-2:],demo['c620_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
2020-10-09,166.141164,163.349827
2020-10-09,173.129485,187.076827


In [20]:
pd.concat([c660_op.iloc[:,-2:],demo['c660_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
2020-10-09,87.499255,93.303923
2020-10-09,85.169942,88.937150


In [21]:
pd.concat([c670_op.iloc[:,-2:],demo['c670_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
2020-10-09,222.828862,463.764844
2020-10-09,182.139600,239.521996
